FIFA'15-23 GAME - Analysis


Prepared by


- Agathiyan K - (045005)


- Hritika Gupta - (045020)


- Mohit Vaidya - (045032)


- Nandini Arya - (045034)

In [ ]:
import pandas as pd, numpy as np # For Data Manipulation
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder # For Encoding Categorical Data [Nominal | Ordinal]
from sklearn.preprocessing import OneHotEncoder # For Creating Dummy Variables of Categorical Data [Nominal]
from sklearn.impute import SimpleImputer, KNNImputer # For Imputation of Missing Data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler # For Rescaling Data
from sklearn.model_selection import train_test_split # For Splitting Data into Training & Testing Sets
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch # For Hierarchical Clustering
from sklearn.cluster import AgglomerativeClustering as agclus, KMeans as kmclus # For Agglomerative & K-Means Clustering
from sklearn.metrics import silhouette_score as sscore, davies_bouldin_score as dbscore # For Clustering Model Evaluation
import scipy.stats as sps # For Probability & Inferential Statistics
import time
import psutil

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Display All Columns of Dataset
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the file ID and construct the file path
file_id = '1ZV-5OFM6glnv9Eji_0mb7CyvPGjR_EPg'
file_path = f'https://drive.google.com/uc?id={file_id}'

# Load the CSV file
df = pd.read_csv(file_path)

# Display the first 5 rows of the DataFrame
df.head(5)

Mounted at /content/drive


<ipython-input-4-b41615e2e735>:11: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,18-09-2014,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,27,24-06-1987,169,67,53.0,La Liga,1.0,241.0,FC Barcelona,CF,10.0,NaN,01-07-2004,2018.0,52,Argentina,1369.0,CF,10.0,Left,3,4,5,Medium/Low,Normal (170-),Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,18-09-2014,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,29,05-02-1985,185,80,53.0,La Liga,1.0,243.0,Real Madrid CF,LW,7.0,NaN,01-07-2009,2018.0,38,Portugal,1354.0,LW,7.0,Right,4,5,5,High/Low,Normal (185+),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,18-09-2014,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,23-01-1984,180,80,19.0,Bundesliga,1.0,21.0,FC Bayern München,SUB,10.0,NaN,28-08-2009,2017.0,34,Netherlands,105035.0,RS,11.0,Left,2,4,5,High/Low,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,18-09-2014,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,32,03-10-1981,195,95,16.0,Ligue 1,1.0,73.0,Paris Saint-Germain,ST,10.0,NaN,01-07-2012,2016.0,46,Sweden,1363.0,ST,10.0,Right,4,4,5,Medium/Low,Normal (185+),Yes,NaN,"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84

In [ ]:
columns_to_drop = ['player_id', 'player_url', 'fifa_update', 'fifa_update_date', 'long_name',
                   'dob', 'league_id',  'league_level', 'club_team_id',
                    'club_position', 'club_jersey_number', 'club_loaned_from',
                   'club_joined_date', 'club_contract_valid_until_year', 'nationality_id',
                    'nation_team_id','nation_jersey_number','player_face_url']

data = df.drop(columns=columns_to_drop)


In [ ]:
data.head(5)


,fifa_version,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,club_name,nationality_name,nation_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,15,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,La Liga,FC Barcelona,Argentina,CF,Left,3,4,5,Medium/Low,Normal (170-),Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,15,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,La Liga,Real Madrid CF,Portugal,LW,Right,4,5,5,High/Low,Normal (185+),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,15,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,Bundesliga,FC Bayern München,Netherlands,RS,Left,2,4,5,High/Low,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,15,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,Ligue 1,Paris Saint-Germain,Sweden,ST,Right,4,4,5,Medium/Low,Normal (185+),Yes,NaN,"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,NaN,25,41,27,13,15,10,9,12,NaN,87+3,87+3,87+3,84+3,86+3,86+3,86+3,84+3,86+3,86+3,86+3,83+3,76+3,76+3,76+3,83+3,61+3,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,15,M. Neuer,GK,90,90,63500000.0,300000.0,28,193,92,Bundesliga,FC Bayern München,Germany,GK,Right,4,1,5,Medium/Medium,Normal (185+),Yes,NaN,NaN,"GK Up for Corners, GK Long Throw, 1-on-1 Rush",NaN,NaN,NaN,NaN,NaN,NaN,25,25,25,42,25,25,25,25,41,31,58,61,43,89,35,42,78,44,83,25,29,30,25,20,37,NaN,25,25,25,87,85,92,90,86,60.0,38+3,38+3,38+3,36+3,37+3,37+3,37+3,36+3,36+3,36+3,36+3,38+3,36+3,36+3,36+3,38+3,36+3,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3


In [ ]:
df1 = data.sample(n=25000, random_state=45005)

In [ ]:
# Create a new DataFrame as a copy of the original
df2 = df1.copy()

# Function to split columns into multiple columns
def split_column(df, column_name):
    if column_name in df.columns:
        split_df = df[column_name].str.split(', ', expand=True)
        split_df.columns = [f'{column_name}_{i+1}' for i in range(split_df.shape[1])]
        return pd.concat([df, split_df], axis=1).drop(columns=[column_name])


# Apply the function to 'player_positions', 'player_tags', and 'player_traits'
df2= split_column(df2, 'player_positions')
df2= split_column(df2, 'player_tags')
df2 = split_column(df2, 'player_traits')

# Check the updated DataFrame
df2.head(5)

,fifa_version,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,club_name,nationality_name,nation_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_positions_1,player_positions_2,player_positions_3,player_positions_4,player_tags_1,player_tags_2,player_tags_3,player_tags_4,player_tags_5,player_tags_6,player_tags_7,player_traits_1,player_traits_2,player_traits_3,player_traits_4,player_traits_5,player_traits_6,player_traits_7,player_traits_8
13527,15,A. Tahiri,57,69,120000.0,2000.0,19,179,61,Jupiler Pro League,Lierse SK,Belgium,NaN,Right,3,3,1,Medium/Medium,Lean (170-185),No,NaN,69.0,37.0,51.0,66.0,24.0,33.0,47,33,27,55,42,66,43,39,51,65,72,67,75,52,77,37,55,38,29,43,32,24,41,52,40,NaN,25,23,22,9,13,12,13,9,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13,RW,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66794,18,W. De Havilland,56,63,110000.0,2000.0,22,183,80,League Two,Wycombe Wanderers,England,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,215000.0,64.0,35.0,45.0,48.0,55.0,61.0,39,31,57,49,37,45,35,24,47,45,62,65,58,54,64,40,63,56,64,38,57,43,32,49,40,53.0,55,60,57,10,7,7,10,7,NaN,45+1,45+1,45+1,46,45,45,45,46,46+1,46+1,46+1,47+1,47+1,47+1,47+1,47+1,51+1,52+1,52+1,52+1,51+1,53+1,55+1,55+1,55+1,53+1,13+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36038,17,Brenito Duarte,73,73,1700000.0,1000.0,32,185,80,Rest of World,Joinville,Brazil,NaN,Left,3,2,1,Medium/Medium,Normal (185+),No,NaN,74.0,43.0,58.0,57.0,71.0,73.0,53,33,72,64,61,45,44,50,76,71,71,77,73,62,62,60,80,58,81,44,71,70,30,45,57,57.0,65,75,75,8,15,15,8,10,NaN,55+1,55+1,55+1,54,53,53,53,54,55+1,55+1,55+1,56+1,60+1,60+1,60+1,56+1,66+1,68+1,68+1,68+1,66+1,68+1,72+1,72+1,72+1,68+1,16+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153356,23,A. Álvarez Wallace,65,74,1500000.0,500.0,21,179,76,Primera Division,Peñarol,Uruguay,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,3500000.0,62.0,52.0,58.0,59.0,61.0,63.0,39,54,44,65,52,51,51,64,69,65,63,62,67,67,61,39,51,72,65,61,48,64,50,58,53,60.0,61,64,59,7,8,16,11,7,NaN,55+2,55+2,55+2,57,57,57,57,57,60+2,60+2,60+2,58+2,63+2,63+2,63+2,58+2,61+2,64+2,64+2,64+2,61+2,61+2,60+2,60+2,60+2,61+2,15+2,CDM,CM,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Long Shot Taker (AI),None,None,None,None,None,None,None
48741,17,S. Griffin,56,66,140000.0,3000.0,21,180,65,Championship,Reading,Republic of Ireland,NaN,Left,3,2,1,Medium/Medium,Lean (170-185),No,NaN,68.0,31.0,47.0,54.0,54.0,54.0,50,32,50,53,31,52,34,32,48,49,67,68,71,54,69,28,59,63,50,26,53,50,51,40,42,46.0,57,54,53,10,14,9,10,11,NaN,46+1,46+1,46+1,50,48,48,48,50,49+1,49+1,49+1,52+1,49+1,49+1,49+1,52+1,55+1,52+1,52+1,52+1,55+1,55+1,53+1,53+1,53+1,55+1,16+1,LB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Dataset Used : df

df2.info() # Dataframe Information (Provide Information on Missing Data)

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 13527 to 13932
Columns: 108 entries, fifa_version to player_traits_8
dtypes: float64(11), int64(42), object(55)
memory usage: 20.8+ MB


In [ ]:
from unidecode import unidecode

In [ ]:
# Function to remove special characters and handle NaN values
def clean_text(text):
    if pd.isna(text):  # Check if the value is NaN
        return text
    return unidecode(text)

# Apply the cleaning function to the columns
df2['short_name'] = df2['short_name'].apply(clean_text)
df2['club_name'] = df2['club_name'].apply(clean_text)
df2['league_name'] = df2['league_name'].apply(clean_text)
df2['nationality_name'] = df2['nationality_name'].apply(clean_text)

# Display the cleaned data
df2.head()

,fifa_version,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,club_name,nationality_name,nation_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_positions_1,player_positions_2,player_positions_3,player_positions_4,player_tags_1,player_tags_2,player_tags_3,player_tags_4,player_tags_5,player_tags_6,player_tags_7,player_traits_1,player_traits_2,player_traits_3,player_traits_4,player_traits_5,player_traits_6,player_traits_7,player_traits_8
13527,15,A. Tahiri,57,69,120000.0,2000.0,19,179,61,Jupiler Pro League,Lierse SK,Belgium,NaN,Right,3,3,1,Medium/Medium,Lean (170-185),No,NaN,69.0,37.0,51.0,66.0,24.0,33.0,47,33,27,55,42,66,43,39,51,65,72,67,75,52,77,37,55,38,29,43,32,24,41,52,40,NaN,25,23,22,9,13,12,13,9,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13,RW,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66794,18,W. De Havilland,56,63,110000.0,2000.0,22,183,80,League Two,Wycombe Wanderers,England,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,215000.0,64.0,35.0,45.0,48.0,55.0,61.0,39,31,57,49,37,45,35,24,47,45,62,65,58,54,64,40,63,56,64,38,57,43,32,49,40,53.0,55,60,57,10,7,7,10,7,NaN,45+1,45+1,45+1,46,45,45,45,46,46+1,46+1,46+1,47+1,47+1,47+1,47+1,47+1,51+1,52+1,52+1,52+1,51+1,53+1,55+1,55+1,55+1,53+1,13+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36038,17,Brenito Duarte,73,73,1700000.0,1000.0,32,185,80,Rest of World,Joinville,Brazil,NaN,Left,3,2,1,Medium/Medium,Normal (185+),No,NaN,74.0,43.0,58.0,57.0,71.0,73.0,53,33,72,64,61,45,44,50,76,71,71,77,73,62,62,60,80,58,81,44,71,70,30,45,57,57.0,65,75,75,8,15,15,8,10,NaN,55+1,55+1,55+1,54,53,53,53,54,55+1,55+1,55+1,56+1,60+1,60+1,60+1,56+1,66+1,68+1,68+1,68+1,66+1,68+1,72+1,72+1,72+1,68+1,16+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153356,23,A. Alvarez Wallace,65,74,1500000.0,500.0,21,179,76,Primera Division,Penarol,Uruguay,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,3500000.0,62.0,52.0,58.0,59.0,61.0,63.0,39,54,44,65,52,51,51,64,69,65,63,62,67,67,61,39,51,72,65,61,48,64,50,58,53,60.0,61,64,59,7,8,16,11,7,NaN,55+2,55+2,55+2,57,57,57,57,57,60+2,60+2,60+2,58+2,63+2,63+2,63+2,58+2,61+2,64+2,64+2,64+2,61+2,61+2,60+2,60+2,60+2,61+2,15+2,CDM,CM,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Long Shot Taker (AI),None,None,None,None,None,None,None
48741,17,S. Griffin,56,66,140000.0,3000.0,21,180,65,Championship,Reading,Republic of Ireland,NaN,Left,3,2,1,Medium/Medium,Lean (170-185),No,NaN,68.0,31.0,47.0,54.0,54.0,54.0,50,32,50,53,31,52,34,32,48,49,67,68,71,54,69,28,59,63,50,26,53,50,51,40,42,46.0,57,54,53,10,14,9,10,11,NaN,46+1,46+1,46+1,50,48,48,48,50,49+1,49+1,49+1,52+1,49+1,49+1,49+1,52+1,55+1,52+1,52+1,52+1,55+1,55+1,53+1,53+1,53+1,55+1,16+1,LB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Function to sum values in the format 'number+number'
def sum_values(value):
    if pd.isna(value):  # Check for NaN values
        return value
    # Convert value to string to handle both integers and strings
    value = str(value)
    # Split the string by '+', convert to integers, and sum
    parts = value.split('+')
    if len(parts) == 2:  # Ensure there are exactly two parts
        return int(parts[0]) + int(parts[1])
    return value

# Apply the function to each column that needs summing
columns_to_sum = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

for column in columns_to_sum:
    df2[f'{column}_sum'] = df2[column].apply(sum_values)

# Display the updated DataFrame
df2.head(5)

,fifa_version,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,club_name,nationality_name,nation_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_positions_1,player_positions_2,player_positions_3,player_positions_4,player_tags_1,player_tags_2,player_tags_3,player_tags_4,player_tags_5,player_tags_6,player_tags_7,player_traits_1,player_traits_2,player_traits_3,player_traits_4,player_traits_5,player_traits_6,player_traits_7,player_traits_8,ls_sum,st_sum,rs_sum,lw_sum,lf_sum,cf_sum,rf_sum,rw_sum,lam_sum,cam_sum,ram_sum,lm_sum,lcm_sum,cm_sum,rcm_sum,rm_sum,lwb_sum,ldm_sum,cdm_sum,rdm_sum,rwb_sum,lb_sum,lcb_sum,cb_sum,rcb_sum,rb_sum,gk_sum
13527,15,A. Tahiri,57,69,120000.0,2000.0,19,179,61,Jupiler Pro League,Lierse SK,Belgium,NaN,Right,3,3,1,Medium/Medium,Lean (170-185),No,NaN,69.0,37.0,51.0,66.0,24.0,33.0,47,33,27,55,42,66,43,39,51,65,72,67,75,52,77,37,55,38,29,43,32,24,41,52,40,NaN,25,23,22,9,13,12,13,9,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13,RW,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13
66794,18,W. De Havilland,56,63,110000.0,2000.0,22,183,80,League Two,Wycombe Wanderers,England,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,215000.0,64.0,35.0,45.0,48.0,55.0,61.0,39,31,57,49,37,45,35,24,47,45,62,65,58,54,64,40,63,56,64,38,57,43,32,49,40,53.0,55,60,57,10,7,7,10,7,NaN,45+1,45+1,45+1,46,45,45,45,46,46+1,46+1,46+1,47+1,47+1,47+1,47+1,47+1,51+1,52+1,52+1,52+1,51+1,53+1,55+1,55+1,55+1,53+1,13+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,46,46,46,45,45,45,46,47,47,47,48,48,48,48,48,52,53,53,53,52,54,56,56,56,54,14
36038,17,Brenito Duarte,73,73,1700000.0,1000.0,32,185,80,Rest of World,Joinville,Brazil,NaN,Left,3,2,1,Medium/Medium,Normal (185+),No,NaN,74.0,43.0,58.0,57.0,71.0,73.0,53,33,72,64,61,45,44,50,76,71,71,77,73,62,62,60,80,58,81,44,71,70,30,45,57,57.0,65,75,75,8,15,15,8,10,NaN,55+1,55+1,55+1,54,53,53,53,54,55+1,55+1,55+1,56+1,60+1,60+1,60+1,56+1,66+1,68+1,68+1,68+1,66+1,68+1,72+1,72+1,72+1,68+1,16+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,56,56,54,53,53,53,54,56,56,56,57,61,61,61,57,67,69,69,69,67,69,73,73,73,69,17
153356,23,A. Alvarez Wallace,65,74,1500000.0,500.0,21,179,76,Primera Division,Penarol,Uruguay,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,3500000.0,62.0,52.0,58.0,59.0,61.0,63.0,39,54,44,65,52,51,51,64,69,65,63,62,67,67,61,39,51,72,65,61,48,64,50,58,53,60.0,61,64,59,7,8,16,11,7,NaN,55+2,55+2,55+2,57,57,57,57,57,60+2,60+2,60+2,58+2,63+2,63+2,63+2,58+2,61+2,64+2,64+2,64+2,61+2,61+2,60+2,60+2,60+2,61+2,15+2,CDM,CM,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Long Shot Taker (AI),None,None,None,None,None,None,None,57,57,57,57,57,57,57,57,62,62,62,60,65,65,65,60,63,66,66,66,63,63,62,62,62,63,17
48741,17,S. Griffin,56,66,140000.0,3000.0,21,180,65,Championship,Reading,Republic of Ireland,NaN,Left,3,2,1,Medium/Medium,Lean (170-185),No,NaN,68.0,31.0,47.0,54.0,54.0,5

In [ ]:
# List all columns in the DataFrame
columns = df2.columns
print(columns)

Index(['fifa_version', 'short_name', 'overall', 'potential', 'value_eur',
       'wage_eur', 'age', 'height_cm', 'weight_kg', 'league_name',
       ...
       'ldm_sum', 'cdm_sum', 'rdm_sum', 'rwb_sum', 'lb_sum', 'lcb_sum',
       'cb_sum', 'rcb_sum', 'rb_sum', 'gk_sum'],
      dtype='object', length=135)


In [ ]:
# List all columns in the DataFrame without truncation
columns = df2.columns.tolist()
for col in columns:
    print(col)

fifa_version
short_name
overall
potential
value_eur
wage_eur
age
height_cm
weight_kg
league_name
club_name
nationality_name
nation_position
preferred_foot
weak_foot
skill_moves
international_reputation
work_rate
body_type
real_face
release_clause_eur
pace
shooting
passing
dribbling
defending
physic
attacking_crossing
attacking_finishing
attacking_heading_accuracy
attacking_short_passing
attacking_volleys
skill_dribbling
skill_curve
skill_fk_accuracy
skill_long_passing
skill_ball_control
movement_acceleration
movement_sprint_speed
movement_agility
movement_reactions
movement_balance
power_shot_power
power_jumping
power_stamina
power_strength
power_long_shots
mentality_aggression
mentality_interceptions
mentality_positioning
mentality_vision
mentality_penalties
mentality_composure
defending_marking_awareness
defending_standing_tackle
defending_sliding_tackle
goalkeeping_diving
goalkeeping_handling
goalkeeping_kicking
goalkeeping_positioning
goalkeeping_reflexes
goalkeeping_speed
ls
st
rs

In [ ]:
# Check for NaN values in the dataset
nan_summary = df2.isna().sum()

# Print columns with NaN values
nan_columns = nan_summary[nan_summary > 0]
print(nan_columns)

value_eur                326
wage_eur                 280
league_name              281
club_name                281
nation_position        23547
release_clause_eur      8878
pace                    2721
shooting                2721
passing                 2721
dribbling               2721
defending               2721
physic                  2721
mentality_composure     5002
goalkeeping_speed      22279
player_positions_2     13226
player_positions_3     21580
player_positions_4     24729
player_tags_1          23017
player_tags_2          24656
player_tags_3          24913
player_tags_4          24963
player_tags_5          24986
player_tags_6          24991
player_tags_7          24999
player_traits_1        13563
player_traits_2        19431
player_traits_3        22700
player_traits_4        24169
player_traits_5        24744
player_traits_6        24921
player_traits_7        24990
player_traits_8        24998
dtype: int64


In [ ]:
# Check for empty cells in the dataset
empty_cells_summary = df2.apply(lambda x: x.eq('').sum())

# Print columns with empty cells
empty_cells_columns = empty_cells_summary[empty_cells_summary > 0]
print(empty_cells_columns)

Series([], dtype: int64)


In [ ]:
# Create an index column
df2['index'] = range(1, len(df2) + 1)

# Reorder columns to place the index column first
cols = ['index'] + [col for col in df2.columns if col != 'index']
df2 = df2[cols]

# Display the updated DataFrame
df2.head()

,index,fifa_version,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,club_name,nationality_name,nation_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_positions_1,player_positions_2,player_positions_3,player_positions_4,player_tags_1,player_tags_2,player_tags_3,player_tags_4,player_tags_5,player_tags_6,player_tags_7,player_traits_1,player_traits_2,player_traits_3,player_traits_4,player_traits_5,player_traits_6,player_traits_7,player_traits_8,ls_sum,st_sum,rs_sum,lw_sum,lf_sum,cf_sum,rf_sum,rw_sum,lam_sum,cam_sum,ram_sum,lm_sum,lcm_sum,cm_sum,rcm_sum,rm_sum,lwb_sum,ldm_sum,cdm_sum,rdm_sum,rwb_sum,lb_sum,lcb_sum,cb_sum,rcb_sum,rb_sum,gk_sum
13527,1,15,A. Tahiri,57,69,120000.0,2000.0,19,179,61,Jupiler Pro League,Lierse SK,Belgium,NaN,Right,3,3,1,Medium/Medium,Lean (170-185),No,NaN,69.0,37.0,51.0,66.0,24.0,33.0,47,33,27,55,42,66,43,39,51,65,72,67,75,52,77,37,55,38,29,43,32,24,41,52,40,NaN,25,23,22,9,13,12,13,9,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13,RW,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13
66794,2,18,W. De Havilland,56,63,110000.0,2000.0,22,183,80,League Two,Wycombe Wanderers,England,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,215000.0,64.0,35.0,45.0,48.0,55.0,61.0,39,31,57,49,37,45,35,24,47,45,62,65,58,54,64,40,63,56,64,38,57,43,32,49,40,53.0,55,60,57,10,7,7,10,7,NaN,45+1,45+1,45+1,46,45,45,45,46,46+1,46+1,46+1,47+1,47+1,47+1,47+1,47+1,51+1,52+1,52+1,52+1,51+1,53+1,55+1,55+1,55+1,53+1,13+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,46,46,46,45,45,45,46,47,47,47,48,48,48,48,48,52,53,53,53,52,54,56,56,56,54,14
36038,3,17,Brenito Duarte,73,73,1700000.0,1000.0,32,185,80,Rest of World,Joinville,Brazil,NaN,Left,3,2,1,Medium/Medium,Normal (185+),No,NaN,74.0,43.0,58.0,57.0,71.0,73.0,53,33,72,64,61,45,44,50,76,71,71,77,73,62,62,60,80,58,81,44,71,70,30,45,57,57.0,65,75,75,8,15,15,8,10,NaN,55+1,55+1,55+1,54,53,53,53,54,55+1,55+1,55+1,56+1,60+1,60+1,60+1,56+1,66+1,68+1,68+1,68+1,66+1,68+1,72+1,72+1,72+1,68+1,16+1,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,56,56,54,53,53,53,54,56,56,56,57,61,61,61,57,67,69,69,69,67,69,73,73,73,69,17
153356,4,23,A. Alvarez Wallace,65,74,1500000.0,500.0,21,179,76,Primera Division,Penarol,Uruguay,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,3500000.0,62.0,52.0,58.0,59.0,61.0,63.0,39,54,44,65,52,51,51,64,69,65,63,62,67,67,61,39,51,72,65,61,48,64,50,58,53,60.0,61,64,59,7,8,16,11,7,NaN,55+2,55+2,55+2,57,57,57,57,57,60+2,60+2,60+2,58+2,63+2,63+2,63+2,58+2,61+2,64+2,64+2,64+2,61+2,61+2,60+2,60+2,60+2,61+2,15+2,CDM,CM,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Long Shot Taker (AI),None,None,None,None,None,None,None,57,57,57,57,57,57,57,57,62,62,62,60,65,65,65,60,63,66,66,66,63,63,62,62,62,63,17
48741,5,17,S. Griffin,56,66,140000.0,3000.0,21,180,65,Championship,Reading,Republic of Ireland,NaN,Left,3,2,1,Medium/Medium,Lean (170-185),No,NaN,68.0,31.0,

In [ ]:
# List of columns to drop
columns_to_drop1 = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam',
                    'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
                    'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

# Drop the specified columns from the DataFrame
df2 = df2.drop(columns=columns_to_drop1)

# Display the updated DataFrame
df2.head()

,index,fifa_version,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_name,club_name,nationality_name,nation_position,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,player_positions_1,player_positions_2,player_positions_3,player_positions_4,player_tags_1,player_tags_2,player_tags_3,player_tags_4,player_tags_5,player_tags_6,player_tags_7,player_traits_1,player_traits_2,player_traits_3,player_traits_4,player_traits_5,player_traits_6,player_traits_7,player_traits_8,ls_sum,st_sum,rs_sum,lw_sum,lf_sum,cf_sum,rf_sum,rw_sum,lam_sum,cam_sum,ram_sum,lm_sum,lcm_sum,cm_sum,rcm_sum,rm_sum,lwb_sum,ldm_sum,cdm_sum,rdm_sum,rwb_sum,lb_sum,lcb_sum,cb_sum,rcb_sum,rb_sum,gk_sum
13527,1,15,A. Tahiri,57,69,120000.0,2000.0,19,179,61,Jupiler Pro League,Lierse SK,Belgium,NaN,Right,3,3,1,Medium/Medium,Lean (170-185),No,NaN,69.0,37.0,51.0,66.0,24.0,33.0,47,33,27,55,42,66,43,39,51,65,72,67,75,52,77,37,55,38,29,43,32,24,41,52,40,NaN,25,23,22,9,13,12,13,9,NaN,RW,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,45,45,57,49,49,49,57,54,54,54,56,49,49,49,56,43,41,41,41,43,36,31,31,31,36,13
66794,2,18,W. De Havilland,56,63,110000.0,2000.0,22,183,80,League Two,Wycombe Wanderers,England,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,215000.0,64.0,35.0,45.0,48.0,55.0,61.0,39,31,57,49,37,45,35,24,47,45,62,65,58,54,64,40,63,56,64,38,57,43,32,49,40,53.0,55,60,57,10,7,7,10,7,NaN,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,46,46,46,45,45,45,46,47,47,47,48,48,48,48,48,52,53,53,53,52,54,56,56,56,54,14
36038,3,17,Brenito Duarte,73,73,1700000.0,1000.0,32,185,80,Rest of World,Joinville,Brazil,NaN,Left,3,2,1,Medium/Medium,Normal (185+),No,NaN,74.0,43.0,58.0,57.0,71.0,73.0,53,33,72,64,61,45,44,50,76,71,71,77,73,62,62,60,80,58,81,44,71,70,30,45,57,57.0,65,75,75,8,15,15,8,10,NaN,CB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,56,56,54,53,53,53,54,56,56,56,57,61,61,61,57,67,69,69,69,67,69,73,73,73,69,17
153356,4,23,A. Alvarez Wallace,65,74,1500000.0,500.0,21,179,76,Primera Division,Penarol,Uruguay,NaN,Right,3,2,1,Medium/Medium,Normal (170-185),No,3500000.0,62.0,52.0,58.0,59.0,61.0,63.0,39,54,44,65,52,51,51,64,69,65,63,62,67,67,61,39,51,72,65,61,48,64,50,58,53,60.0,61,64,59,7,8,16,11,7,NaN,CDM,CM,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Long Shot Taker (AI),None,None,None,None,None,None,None,57,57,57,57,57,57,57,57,62,62,62,60,65,65,65,60,63,66,66,66,63,63,62,62,62,63,17
48741,5,17,S. Griffin,56,66,140000.0,3000.0,21,180,65,Championship,Reading,Republic of Ireland,NaN,Left,3,2,1,Medium/Medium,Lean (170-185),No,NaN,68.0,31.0,47.0,54.0,54.0,54.0,50,32,50,53,31,52,34,32,48,49,67,68,71,54,69,28,59,63,50,26,53,50,51,40,42,46.0,57,54,53,10,14,9,10,11,NaN,LB,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,47,47,50,48,48,48,50,50,50,50,53,50,50,50,53,56,53,53,53,56,56,54,54,54,56,17


In [ ]:
# Save the DataFrame to a CSV file
df2.to_csv('final.csv', index=False)

In [ ]:
# Convert columns to numeric
numeric_columns = [
    'ls_sum', 'st_sum', 'rs_sum', 'lw_sum', 'lf_sum', 'cf_sum', 'rf_sum', 'rw_sum',
    'lam_sum', 'cam_sum', 'ram_sum', 'lm_sum', 'lcm_sum', 'cm_sum', 'rcm_sum', 'rm_sum',
    'lwb_sum', 'ldm_sum', 'cdm_sum', 'rdm_sum', 'rwb_sum', 'lb_sum', 'lcb_sum', 'cb_sum',
    'rcb_sum', 'rb_sum', 'gk_sum'
]

for column in numeric_columns:
    df2[column] = pd.to_numeric(df2[column], errors='coerce')


# Verify the data types
df2[numeric_columns].dtypes

,0
ls_sum,float64
st_sum,float64
rs_sum,float64
lw_sum,float64
lf_sum,float64
cf_sum,float64
rf_sum,float64
rw_sum,float64
lam_sum,float64
cam_sum,float64


In [ ]:
# Correct the column classification
df2 = df2.astype({
    'ls_sum': 'float64', 'st_sum': 'float64', 'rs_sum': 'float64', 'lw_sum': 'float64',
    'lf_sum': 'float64', 'cf_sum': 'float64', 'rf_sum': 'float64', 'rw_sum': 'float64',
    'lam_sum': 'float64', 'cam_sum': 'float64', 'ram_sum': 'float64', 'lm_sum': 'float64',
    'lcm_sum': 'float64', 'cm_sum': 'float64', 'rcm_sum': 'float64', 'rm_sum': 'float64',
    'lwb_sum': 'float64', 'ldm_sum': 'float64', 'cdm_sum': 'float64', 'rdm_sum': 'float64',
    'rwb_sum': 'float64', 'lb_sum': 'float64', 'lcb_sum': 'float64', 'cb_sum': 'float64',
    'rcb_sum': 'float64', 'rb_sum': 'float64', 'gk_sum': 'int64'
})
df2.dtypes

,0
index,int64
fifa_version,int64
short_name,object
overall,int64
potential,int64
...,...
lcb_sum,float64
cb_sum,float64
rcb_sum,float64
rb_sum,float64


In [ ]:
!pip install dash
!pip install dash-bootstrap-components
!pip install plotly

!pip install dash plotly pandas
!python app.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 3.5 MB/s eta 0:00:00
python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

# Define wage ranges
wage_ranges = [
    (0, 100000), (100001, 200000), (200001, 300000), (300001, 400000),
    (400001, 500000), (500001, 600000), (600001, 700000), (700001, 800000),
    (800001, 900000), (900001, 1000000)
]

# Create wage range labels
wage_range_labels = [f"{low:,} - {high:,}" for low, high in wage_ranges]

# Sample DataFrame for demonstration (use your df2 here)
# df2 = pd.read_csv('your_data.csv')  # Ensure to use your actual data

# Create options for player tags
player_tags_columns = ['player_tags_1', 'player_tags_2', 'player_tags_3',
                       'player_tags_4', 'player_tags_5', 'player_tags_6', 'player_tags_7']
player_tags = sorted(set(tag for col in player_tags_columns for tag in df2[col].dropna().unique()))
player_tags_options = [{'label': tag, 'value': tag} for tag in player_tags]

# Layout of the app
app.layout = html.Div([
    html.H1("FIFA Player Stats Dashboard"),

    # Dropdown for selecting FIFA version
    dcc.Dropdown(
        id='fifa-version-dropdown',
        options=[{'label': version, 'value': version} for version in sorted(df2['fifa_version'].unique())],
        placeholder="Select FIFA Version"
    ),

    # First Visualization: Player Position Bar Chart
    html.Div([
        html.H2("Player Stats by Position"),

        # Dropdown for selecting player positions
        dcc.Dropdown(
            id='player-positions-dropdown',
            placeholder="Select Player Position"
        ),

        # Dropdown for selecting league name
        dcc.Dropdown(
            id='league-name-dropdown',
            placeholder="Select League Name"
        ),

        # Dropdown for selecting club name
        dcc.Dropdown(
            id='club-name-dropdown',
            placeholder="Select Club Name"
        ),

        # Bar chart to display player stats
        dcc.Graph(id='player-bar-chart')
    ]),

    # Second Visualization: Wage Range Treemap
    html.Div([
        html.H2("Player Stats by Wage Range"),

        # Dropdown for selecting wage range
        dcc.Dropdown(
            id='wage-range-dropdown',
            options=[{'label': label, 'value': i} for i, label in enumerate(wage_range_labels)],
            placeholder="Select Wage Range"
        ),

        # Treemap for player positions
        dcc.Graph(id='player-treemap'),

        # Table to display player details
        html.Div(id='player-details')
    ]),

    # Third Visualization: Player Tags Bar Chart
    html.Div([
        html.H2("Player Stats by Tags"),

        # Dropdown for selecting player tags
        dcc.Dropdown(
            id='player-tags-dropdown',
            options=player_tags_options,
            multi=True,
            placeholder="Select Player Tags"
        ),

        # Dropdown for selecting player names
        dcc.Dropdown(
            id='player-names-dropdown',
            multi=True,
            placeholder="Select Player Names"
        ),

        # Bar chart to display player stats by tags
        dcc.Graph(id='player-tags-bar-chart')
    ])
])

# Callback to update player positions based on selected FIFA version
@app.callback(
    Output('player-positions-dropdown', 'options'),
    Input('fifa-version-dropdown', 'value')
)
def update_positions(fifa_version):
    if fifa_version is None:
        return []
    positions = df2[df2['fifa_version'] == fifa_version]['player_positions_1'].dropna().unique()
    return [{'label': position, 'value': position} for position in sorted(positions)]

# Callback to update league names based on selected player position
@app.callback(
    Output('league-name-dropdown', 'options'),
    [Input('fifa-version-dropdown', 'value'),
     Input('player-positions-dropdown', 'value')]
)
def update_leagues(fifa_version, player_position):
    if fifa_version is None or player_position is None:
        return []
    leagues = df2[(df2['fifa_version'] == fifa_version) &
                  (df2['player_positions_1'] == player_position)]['league_name'].dropna().unique()
    if len(leagues) == 0:
        return [{'label': 'No Leagues Available', 'value': 'N/A'}]
    return [{'label': league, 'value': league} for league in sorted(leagues)]

# Callback to update club names based on selected league name
@app.callback(
    Output('club-name-dropdown', 'options'),
    [Input('fifa-version-dropdown', 'value'),
     Input('player-positions-dropdown', 'value'),
     Input('league-name-dropdown', 'value')]
)
def update_clubs(fifa_version, player_position, league_name):
    if fifa_version is None or player_position is None or league_name is None or league_name == 'N/A':
        return []
    clubs = df2[(df2['fifa_version'] == fifa_version) &
                (df2['player_positions_1'] == player_position) &
                (df2['league_name'] == league_name)]['club_name'].dropna().unique()
    if len(clubs) == 0:
        return [{'label': 'No Clubs Available', 'value': 'N/A'}]
    return [{'label': club, 'value': club} for club in sorted(clubs)]

# Callback to update the player stats bar chart based on selected club name
@app.callback(
    Output('player-bar-chart', 'figure'),
    [Input('fifa-version-dropdown', 'value'),
     Input('player-positions-dropdown', 'value'),
     Input('league-name-dropdown', 'value'),
     Input('club-name-dropdown', 'value')]
)
def update_player_bar_chart(fifa_version, player_position, league_name, club_name):
    if fifa_version is None or player_position is None or league_name is None or club_name is None or club_name == 'N/A':
        return {}

    filtered_df = df2[(df2['fifa_version'] == fifa_version) &
                      (df2['player_positions_1'] == player_position) &
                      (df2['league_name'] == league_name) &
                      (df2['club_name'] == club_name)]

    if filtered_df.empty:
        return {}

    fig = px.bar(filtered_df, x='short_name', y=['overall', 'potential', 'value_eur', 'age'],
                 barmode='group', labels={'value': 'Value (€)'})

    return fig

# Callback to update treemap based on selected FIFA version and wage range
@app.callback(
    Output('player-treemap', 'figure'),
    [Input('fifa-version-dropdown', 'value'),
     Input('wage-range-dropdown', 'value')]
)
def update_treemap(fifa_version, wage_range_index):
    if fifa_version is None or wage_range_index is None:
        return {}

    # Filter by FIFA version
    filtered_df = df2[df2['fifa_version'] == fifa_version]

    # Apply wage range filter
    low, high = wage_ranges[wage_range_index]
    filtered_df = filtered_df[(filtered_df['wage_eur'] >= low) & (filtered_df['wage_eur'] <= high)]

    if filtered_df.empty:
        return {}

    # Create treemap
    fig = px.treemap(
        filtered_df,
        path=['player_positions_1'],
        values='wage_eur',
        title='Player Positions Treemap'
    )

    return fig

# Callback to update player details based on selected country and treemap selection
@app.callback(
    Output('player-details', 'children'),
    [Input('player-treemap', 'clickData'),
     Input('fifa-version-dropdown', 'value'),
     Input('wage-range-dropdown', 'value')]
)
def update_player_details(clickData, fifa_version, wage_range_index):
    if clickData is None or fifa_version is None or wage_range_index is None:
        return "Select a player position in the treemap to see details."

    # Get the selected player position
    player_position = clickData['points'][0]['id']

    # Filter by FIFA version and wage range
    low, high = wage_ranges[wage_range_index]
    filtered_df = df2[(df2['fifa_version'] == fifa_version) &
                      (df2['wage_eur'] >= low) &
                      (df2['wage_eur'] <= high) &
                      (df2['player_positions_1'] == player_position)]

    if filtered_df.empty:
        return "No players found in this filter."

    # Create a table of player details
    table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in ['Short Name', 'Position', 'Nationality', 'Club Name', 'Wage EUR', 'Value EUR', 'Overall', 'Potential', 'Age']])),
        html.Tbody([
            html.Tr([
                html.Td(row['short_name']),
                html.Td(row['player_positions_1']),
                html.Td(row['nationality_name']),
                html.Td(row['club_name']),
                html.Td(f"{row['wage_eur']:,}"),
                html.Td(f"{row['value_eur']:,}"),
                html.Td(row['overall']),
                html.Td(row['potential']),
                html.Td(row['age'])
            ]) for _, row in filtered_df.iterrows()
        ])
    ])

    return table

# Callback to update player names dropdown based on selected player tags
@app.callback(
    Output('player-names-dropdown', 'options'),
    [Input('player-tags-dropdown', 'value')]
)
def update_player_names(selected_tags):
    if not selected_tags:
        return []

    filtered_df = df2[df2[player_tags_columns].isin(selected_tags).any(axis=1)]
    players = filtered_df['short_name'].unique()
    return [{'label': player, 'value': player} for player in sorted(players)]

# Callback to update player stats bar chart based on selected player names
@app.callback(
    Output('player-tags-bar-chart', 'figure'),
    [Input('player-names-dropdown', 'value')]
)
def update_player_tags_bar_chart(selected_players):
    if not selected_players:
        return {}

    filtered_df = df2[df2['short_name'].isin(selected_players)]

    fig = px.bar(filtered_df, x='short_name', y=['overall', 'potential', 'value_eur', 'age'],
                 barmode='group', labels={'value': 'Value (€)'})

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# ALL THREE

<IPython.core.display.Javascript object>